In [ ]:
print("all_ok")

In [ ]:
from langchain_groq import ChatGroq

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
llm=ChatGroq(model="llama-3.3-70b-versatile")

In [ ]:
print(llm.invoke("What is the capital of France?").content)

In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

In [ ]:
embedding_model=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [ ]:
doc_vector=embedding_model.embed_query("What is the capital of France?")

## 1. Data Ingestion

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [ ]:
import os

In [ ]:
os.getcwd()

In [ ]:
file_path=os.path.join(os.getcwd(), "data", "sample.pdf")

In [ ]:
loader=PyPDFLoader(file_path)

In [ ]:
documents=loader.load()

In [ ]:
documents

In [ ]:
len(documents)

### this is a experimental thing there is no deterministic way to split the text

In [ ]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150,
    length_function=len
)

In [ ]:
docs=text_splitter.split_documents(documents)

In [ ]:
len(docs)

In [ ]:
docs[0]

In [ ]:
docs[0].metadata

In [ ]:
docs[1].metadata

In [ ]:
0,1,2,3
page_lable=4
actual page is 4 

page_index=3
page_lable=4

In [ ]:
docs[100].metadata

In [ ]:
docs[200].metadata

In [ ]:
docs[0].page_content

In [ ]:
from langchain_community.vectorstores import FAISS

In [ ]:
docs[0].page_content

In [ ]:
doc_vector=embedding_model.embed_documents(docs[0].page_content)

In [ ]:
len(embedding_model.embed_documents(docs[0].page_content)[0])

In [ ]:
vectorstore=FAISS.from_documents(docs, embedding_model)

In [ ]:
token->words

chunk--> it is collection of words(token)[characters]

1. in memory(faiss is in memory vector store,chroma)
2. on disk storage(faiss you can persist over the disk,chroma)
3. cloud storage(cloud variant of faiss is not available)(pinecone,weaviate,milvus,mongodbvectorsearch,astradb)

## This is a Retrieval proceesss

means from the vectordatabase we are going to fetch or retrive or rank the most appropriate k result

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.")

In [ ]:
relevant_doc

In [ ]:
relevant_doc[0].page_content

In [ ]:
relevant_doc[1].page_content

In [ ]:
relevant_doc[2].page_content

In [ ]:
relevant_doc[3].page_content

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=10)

In [ ]:
relevant_doc

In [ ]:
relevant_doc=vectorstore.similarity_search("llama2 finetuning benchmark experiments.",k=1)

In [ ]:
relevant_doc

In [ ]:
# retriever=vectorstore.as_retriever()

### you can explore about keyword filtering

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})

In [ ]:
retriever.invoke("llama2 finetuning benchmark experiments.")

## Question: user question
## Context: based on the question retrieving the info from the vector database

In [ ]:
prompt_template = """
        Answer the question based on the context provided below. 
        If the context does not contain sufficient information, respond with: 
        "I do not have enough information about this."

        Context: {context}

        Question: {question}

        Answer:"""


In [ ]:
from langchain.prompts import PromptTemplate

In [ ]:
prompt=PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)

In [ ]:
prompt

PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='\n        Answer the question based on the context provided below. \n        If the context does not contain sufficient information, respond with: \n        "I do not have enough information about this."\n\n        Context: {context}\n\n        Question: {question}\n\n        Answer:')

In [ ]:
from langchain_core.output_parsers import StrOutputParser

In [ ]:
parser=StrOutputParser()

In [ ]:
def format_docs(docs):
    return "\n\n".join([doc.page_content for doc in docs])

In [ ]:
from langchain_core.runnables import RunnablePassthrough

In [ ]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("tell  me about the llama2 finetuning benchmark experiments?")

In [ ]:
rag_chain.invoke("can you tell me Scaling trends for the reward model?")

# One Small task

### Take 10 pdfs keep it in same directory and create RAG on top of it.

In next class will discuss about the(will start with the modular coding)
1. exception module
2. logger module
3. doc analyser
4. doc compare
5. utils and config
   
2 class

2 more class api and other module

2 more class for deployment